In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from pandas import DataFrame, concat
from pandas import DataFrame
import matplotlib.pyplot as plt
from sklearn.experimental import enable_iterative_imputer
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyClassifier

# Preparation

In [19]:
climb = pd.read_csv('data/climb.csv', sep=";")

climb = climb.drop(columns=['subject_id'])
# drop everything after column sgrq_total_score_visit2
climb = climb.iloc[:, :climb.columns.get_loc('sgrq_total_score_visit2') + 1]
# drop adverse_irritation	adverse_allergic	adverse_taste	adverse_teeth	adverse_irritation_2	adverse_allergic_2	adverse_taste_2	adverse_teeth_2
climb = climb.drop(columns=['adverse_irritation', 'adverse_allergic', 'adverse_taste', 'adverse_teeth', 'adverse_irritation_2', 'adverse_allergic_2', 'adverse_taste_2', 'adverse_teeth_2', 'crp', 'fibrinogen', 'leukocyte'
])

# remove 'smoked' because everybody smoked, remove season and withdrawn because irrelevant
climb = climb.drop(columns=['smoked', 'season', 'withdrawn', 'randassign'])

# # plot histogram of all variables
# climb.hist(figsize=(20, 20))

# # plot boxplot
# climb.plot(kind='box', subplots=True, layout=(5, 5), sharex=False, sharey=False, figsize=(20, 20))

# convert age categories to categorical data: 40-49 = 0, 50-59 = 1, 60-69 = 2, 70-79 = 3, 80-84 = 4
climb['age range'] = climb['age range'].replace(['40-49', '50-59', '60-69', '70-79', '80-84'], [0, 1, 2, 3, 4])


# impute missing values with median
climb['sgrq_symptoms_score_visit2'] = climb['sgrq_symptoms_score_visit2'].fillna(climb['sgrq_symptoms_score_visit2'].median())
climb['sgrq_activity_score_visit2'] = climb['sgrq_activity_score_visit2'].fillna(climb['sgrq_activity_score_visit2'].median())
climb['sgrq_impacts_score_visit2'] = climb['sgrq_impacts_score_visit2'].fillna(climb['sgrq_impacts_score_visit2'].median())
climb['sgrq_total_score_visit2'] = climb['sgrq_total_score_visit2'].fillna(climb['sgrq_total_score_visit2'].median())

# scaling 
scaler = MinMaxScaler()
target = climb['numb_exac_12month']
climb = climb.drop(columns=['numb_exac_12month'])
climb = pd.DataFrame(scaler.fit_transform(climb), columns=climb.columns)
climb = pd.concat([climb, target], axis=1)

# target variable

# binarize target: if hospitalized_num_copd > 0, then 1, else 0
# climb['hospitalized_num_copd'] = np.where(climb['hospitalized_num_copd'] > 0, 1, 0)
# print(climb['hospitalized_num_copd'].value_counts())


In [20]:
# classification
X = climb.drop(columns=['numb_exac_12month'])
y = climb['numb_exac_12month']

# create train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
models


100%|██████████| 29/29 [00:00<00:00, 67.12it/s]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LabelPropagation,0.44,0.44,None,0.35,0.01
LabelSpreading,0.44,0.44,None,0.35,0.01
GaussianNB,0.33,0.38,None,0.24,0.01
ExtraTreeClassifier,0.44,0.27,None,0.41,0.01
LGBMClassifier,0.44,0.25,None,0.27,0.01
DummyClassifier,0.44,0.25,None,0.27,0.01
RandomForestClassifier,0.33,0.21,None,0.29,0.09
KNeighborsClassifier,0.33,0.19,None,0.22,0.01
NearestCentroid,0.22,0.15,None,0.21,0.01
